In [1]:
import FinanceDataReader as fdr
import sqlite3
from tqdm import tqdm
import pandas as pd

In [6]:
symbol = pd.read_csv('./code_list_2018.txt', header= None)
symbol

,0
0,50120
1,95340
2,67290
3,24120
4,60720
...,...
1165,210
1166,4840
1167,155660
1168,78930


In [7]:
symbol.columns = ['Code']

In [8]:
symbol['Code'] = symbol['Code'].apply(lambda x: str(x).zfill(6))
symbol

,Code
0,050120
1,095340
2,067290
3,024120
4,060720
...,...
1165,000210
1166,004840
1167,155660
1168,078930


In [9]:
lst_code = symbol['Code'].tolist()
lst_code[:5]

['050120', '095340', '067290', '024120', '060720']

In [11]:
df = pd.DataFrame()

start_date = '2018-01-01'
end_date = '2021-12-31'


for code in tqdm(lst_code):
    data = fdr.DataReader(code, start=start_date, end=end_date)
    data.reset_index(inplace = True)
    data['Code'] = code
    df = pd.concat([df, data], axis =0)
    

display(df, df.shape) 

100%|██████████████████████████████████████████████████████████████████████████████| 1170/1170 [04:46<00:00,  4.08it/s]


,Date,Open,High,Low,Close,Volume,Change,Code
0,2018-01-02,10250,12050,10150,11800,26086769,0.145631,050120
1,2018-01-03,11950,12450,10900,11750,20460474,-0.004237,050120
2,2018-01-04,11850,14150,11600,12600,60663854,0.072340,050120
3,2018-01-05,12800,13200,12000,12200,13935258,-0.031746,050120
4,2018-01-08,12450,13400,12350,12850,16471707,0.053279,050120
...,...,...,...,...,...,...,...,...
981,2021-12-24,2375,2385,2350,2360,246202,-0.004219,001250
982,2021-12-27,2365,2370,2320,2320,258195,-0.016949,001250
983,2021-12-28,2335,2350,2290,2315,375153,-0.002155,001250
984,2021-12-29,2310,2360,2295,2355,190140,0.017279,001250


(1153620, 8)

In [13]:
df.columns

Index(['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Change', 'Code'], dtype='object')

In [22]:
# 데이터프레임의 NaN 값을 적절한 값으로 대체 (예: 0 또는 빈 문자열)
df.fillna({'Open': 0, 'High': 0, 'Low': 0, 'Close': 0, 'Volume': 0, 'Change': 0, 'Code': ''}, inplace=True)

# 날짜 형식이 문자열인지 확인하고 변환
df['Date'] = pd.to_datetime(df['Date']).dt.strftime('%Y-%m-%d')

In [23]:
try:
    # SQLite3 연결
    conn = sqlite3.connect('stock_data.db')  # 데이터베이스 파일 경로
    cursor = conn.cursor()

    
    # 테이블 생성 쿼리
    create_table_query = """
    CREATE TABLE IF NOT EXISTS main (
        Date TEXT,
        Open REAL,
        High REAL,
        Low REAL,
        Close REAL,
        Volume INTEGER,
        Change REAL,
        Code TEXT
    )
    """
    
    # 테이블 생성
    cursor.execute(create_table_query)
    
    # 데이터 삽입
    for index, row in df.iterrows():
        cursor.execute(insert_query, (
            row['Date'], 
            float(row['Open']), 
            float(row['High']), 
            float(row['Low']), 
            float(row['Close']), 
            int(row['Volume']), 
            float(row['Change']), 
            str(row['Code'])
        ))

    # 변경사항 저장
    conn.commit()

except sqlite3.Error as err:
    print("Something went wrong: {}".format(err))

finally:
    if conn:
        # 연결 닫기
        cursor.close()
        conn.close()

In [24]:
display(df)

,Date,Open,High,Low,Close,Volume,Change,Code
0,2018-01-02,10250,12050,10150,11800,26086769,0.145631,050120
1,2018-01-03,11950,12450,10900,11750,20460474,-0.004237,050120
2,2018-01-04,11850,14150,11600,12600,60663854,0.072340,050120
3,2018-01-05,12800,13200,12000,12200,13935258,-0.031746,050120
4,2018-01-08,12450,13400,12350,12850,16471707,0.053279,050120
...,...,...,...,...,...,...,...,...
981,2021-12-24,2375,2385,2350,2360,246202,-0.004219,001250
982,2021-12-27,2365,2370,2320,2320,258195,-0.016949,001250
983,2021-12-28,2335,2350,2290,2315,375153,-0.002155,001250
984,2021-12-29,2310,2360,2295,2355,190140,0.017279,001250


In [30]:
sql_query = '''
        SELECT * FROM 
        main
        WHERE Date
        BETWEEN '2018-01-01' AND '2018-12-31'
        '''

stock = pd.read_sql(sql = sql_query , con = conn)
stock

,Date,Open,High,Low,Close,Volume,Change,Code
0,2018-01-02,10250.0,12050.0,10150.0,11800.0,26086769,0.145631,050120
1,2018-01-03,11950.0,12450.0,10900.0,11750.0,20460474,-0.004237,050120
2,2018-01-04,11850.0,14150.0,11600.0,12600.0,60663854,0.072340,050120
3,2018-01-05,12800.0,13200.0,12000.0,12200.0,13935258,-0.031746,050120
4,2018-01-08,12450.0,13400.0,12350.0,12850.0,16471707,0.053279,050120
...,...,...,...,...,...,...,...,...
285475,2018-12-21,2560.0,2570.0,2525.0,2535.0,158297,-0.015534,001250
285476,2018-12-24,2560.0,2560.0,2510.0,2515.0,133433,-0.007890,001250
285477,2018-12-26,2485.0,2485.0,2415.0,2455.0,249575,-0.023857,001250
285478,2018-12-27,2485.0,2490.0,2435.0,2445.0,158527,-0.004073,001250
